In [1]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
con = sqlite3.connect("D:/Data_Practice/final.sqlite")

processed_data = pd.read_sql_query('''SELECT * FROM Reviews''',con)
processed_data.shape
processed_data.head()

final_pos = processed_data[processed_data.Score=='Positive']
final_pos.shape

final_neg = processed_data[processed_data.Score=='Negative']
final_neg.shape

final_pos_5000 = final_pos[0:5000]
final_pos_5000.shape
print(final_pos_5000.head(5))

final_neg_5000 = final_neg[0:5000]
final_neg_5000.shape
print(final_neg_5000.head(5))

final_10000 = pd.concat([final_pos_5000 , final_neg_5000],axis=0,ignore_index=True)
final_10000.tail()

sample = final_10000.sort_values(by='Time')
score_10000 = sample['Score']
sample.head()
sample['Score'].value_counts()

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,2))
final_bow = count_vect.fit_transform(sample['Cleaned_Text'].values)
print(final_bow.shape)
#Standardization
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#std_data = scaler.fit_transform(final_bow.todense())

    index      Id   ProductId          UserId                  ProfileName  \
0  138706  150524  0006641040   ACITT7DI6IDDL              shari zychinski   
1  138688  150506  0006641040  A2IW4PEEKO2R0U                        Tracy   
2  138689  150507  0006641040  A1S4A3IQ2MU7V4        sally sue "sally sue"   
3  138690  150508  0006641040     AZGXZ2UUK6X  Catherine Hallberg "(Kate)"   
4  138691  150509  0006641040  A3CMRKGE0P909G                       Teresa   

   HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
0                     0                       0  Positive   939340800   
1                     1                       1  Positive  1194739200   
2                     1                       1  Positive  1191456000   
3                     1                       1  Positive  1076025600   
4                     3                       4  Positive  1018396800   

                                      Summary  \
0                   EVERY book is education

In [2]:
sample['Score'].value_counts()

Negative    5000
Positive    5000
Name: Score, dtype: int64

In [3]:
#SVM
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X = final_bow
Y = score_10000

x_1,x_test,y_1,y_test = train_test_split(X,Y,test_size=0.3,random_state=10)

tuned_parameters = [{'C': [10**-4, 10**-2, 10, 100, 10**3]},{'gamma':[10**-2,10**-1,10,100,1000]}]
model = GridSearchCV(SVC(),tuned_parameters,scoring='accuracy')
model.fit(x_1,y_1)
print(model.best_estimator_)
print(model.score(x_test,y_test))



SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
0.846


In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

clf = NuSVC(nu=0.2)
clf.fit(x_1,y_1)

pred = clf.predict(x_test)

acc = accuracy_score(pred,y_test)*100

print(acc)

84.76666666666667
